In [447]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor  # or RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [448]:
train_test = pd.read_csv("../processing_2/train_test_merged_monthly.csv", parse_dates=['Date'])

In [449]:
# train_test[train_test["Num Parcelle"].isin(['20093', '12251', '22014', '12252', '22012', '12022', '12122'])]['Site'].unique()

In [450]:
train_test

,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,Date,Tonnage,is_train,...,K_cumulee_9,N_cumulee_10,P_cumulee_10,K_cumulee_10,N_cumulee_11,P_cumulee_11,K_cumulee_11,N_cumulee_12,P_cumulee_12,K_cumulee_12
0,20/21,GHARB,Chorf Laghouazi,CLA,CLA1,13030,TR11130,2020-10-01,"269,855",True,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,20/21,GHARB,Chorf Laghouazi,CLA,CLA6,18030,TR11140,2020-10-01,"101605,9951",True,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,20/21,GHARB,Chorf Laghouazi,CLA,CLA12,13010,TR11120,2020-10-01,"170,354",True,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,20/21,GHARB,Chorf Laghouazi,CLA,CLA14,12010,TR11090,2020-10-01,"37891,08567",True,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,20/21,GHARB,Chorf Laghouazi,CLA,CLA14,12020,TR11100,2020-10-01,"45197,56034",True,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,22/23,SOUSS,Ouled Abbou,SPA,SPA4,21071,TR21071,2023-09-01,NaN,False,...,0.324000,0.0,0.0,0.0,0.0,0.0,0.0,0.116095,0.0,0.058046
3896,22/23,SOUSS,Ouled Abbou,SPA,SPA5,21021,TR21021,2023-09-01,NaN,False,...,0.163391,0.0,0.0,0.0,0.0,0.0,0.0,0.119072,0.0,0.059535
3897,22/23,SOUSS,Ouled Abbou,SPA,SPA6,21121,TR21121,2023-09-01,NaN,False,...,0.015223,0.0,0.0,0.0,0.0,0.0,0.0,0.028280,0.0,0.014139
3898,22/23,SOUSS,Ouled Abbou,SPA,SPA7,21081,TR21081,2023-09-01,NaN,False,...,0.587322,0.0,0.0,0.0,0.0,0.0,0.0,0.495950,0.0,0.247975


In [451]:
train_test['Tonnage'] = train_test['Tonnage'].str.replace(',', '.').astype(float)
# train_test['Tonnage'] = train_test['Tonnage'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

In [452]:
train_test['Tonnage'].mean(), train_test['Tonnage'].std()

(20812.88503404854, 38966.13619259874)

In [453]:
# columns_to_sum = ['irrigation_cumulee_' + str(i) for i in range(1, 13)]

# # Compute the row-wise sum of these columns
# row_sums = train_test[columns_to_sum].sum(axis=1)

# # Create a sub-dataframe where the sum is zero
# train_test[row_sums == 0]

In [454]:
for month_index in range(12):
    train_test[f'irrigation_cumulee_{month_index + 1}'] = train_test[f'irrigation_cumulee_{month_index + 1}'].replace(0, np.nan)

In [455]:
for month_index in range(12):
    # print(f'Len irrigation {month_index + 1}:', len(train_test[train_test[f'irrigation_cumulee_{month_index + 1}'] == 0]))
    print(f'Len irrigation {month_index + 1}:', len(train_test[train_test[f'irrigation_cumulee_{month_index + 1}'].isna()]))

Len irrigation 1: 817
Len irrigation 2: 397
Len irrigation 3: 498
Len irrigation 4: 705
Len irrigation 5: 1005
Len irrigation 6: 1254
Len irrigation 7: 1449
Len irrigation 8: 1490
Len irrigation 9: 1532
Len irrigation 10: 1548
Len irrigation 11: 1601
Len irrigation 12: 1659


In [456]:
train_test = train_test.drop('CodeTracabilite', axis=1)
train_test = train_test.drop('Num Parcelle', axis=1)
# Parse dates
# train_test['Date'] = pd.to_datetime(train_test['Date'])
train_test['Year'] = train_test['Date'].dt.year
train_test['Month'] = train_test['Date'].dt.month
train_test['Day'] = train_test['Date'].dt.day

# Drop the original 'Date' column if it's no longer needed
train_test.drop('Campagne', axis=1, inplace=True)
train_test.drop('Day', axis=1, inplace=True)
train_test.drop('Date', axis=1, inplace=True)

## Pipeline

Preprocessing

In [457]:
cat_cols = ['Variete', 'Month', 'Year', 'Region', 'Site', 'Recolte']

# Columns for KNN Imputation
cols_to_impute = ['irrigation_cumulee_1', 'irrigation_cumulee_2', 'irrigation_cumulee_3', 'irrigation_cumulee_4', 'irrigation_cumulee_5', 'irrigation_cumulee_6',
                  'irrigation_cumulee_7', 'irrigation_cumulee_8', 'irrigation_cumulee_9', 'irrigation_cumulee_10', 'irrigation_cumulee_11', 'irrigation_cumulee_12']

# All columns in the dataset
all_cols = train_test.columns.tolist()

# Columns that are neither categorical nor to be imputed (remaining numerical columns)
num_cols = [col for col in all_cols if col not in cat_cols+['is_train']]

In [458]:
num_cols

['Famille',
 'Tonnage',
 'irrigation_cumulee_1',
 'irrigation_cumulee_2',
 'irrigation_cumulee_3',
 'irrigation_cumulee_4',
 'irrigation_cumulee_5',
 'irrigation_cumulee_6',
 'irrigation_cumulee_7',
 'irrigation_cumulee_8',
 'irrigation_cumulee_9',
 'irrigation_cumulee_10',
 'irrigation_cumulee_11',
 'irrigation_cumulee_12',
 'N_cumulee_1',
 'P_cumulee_1',
 'K_cumulee_1',
 'N_cumulee_2',
 'P_cumulee_2',
 'K_cumulee_2',
 'N_cumulee_3',
 'P_cumulee_3',
 'K_cumulee_3',
 'N_cumulee_4',
 'P_cumulee_4',
 'K_cumulee_4',
 'N_cumulee_5',
 'P_cumulee_5',
 'K_cumulee_5',
 'N_cumulee_6',
 'P_cumulee_6',
 'K_cumulee_6',
 'N_cumulee_7',
 'P_cumulee_7',
 'K_cumulee_7',
 'N_cumulee_8',
 'P_cumulee_8',
 'K_cumulee_8',
 'N_cumulee_9',
 'P_cumulee_9',
 'K_cumulee_9',
 'N_cumulee_10',
 'P_cumulee_10',
 'K_cumulee_10',
 'N_cumulee_11',
 'P_cumulee_11',
 'K_cumulee_11',
 'N_cumulee_12',
 'P_cumulee_12',
 'K_cumulee_12']

In [459]:
len(all_cols) - len(cat_cols)

51

In [460]:
# Define the transformer
cat_transformer = OneHotEncoder(sparse=False, handle_unknown='ignore')

# KNNImputer for numerical features
knn_imputer = KNNImputer(n_neighbors=350)

### Inspect the data

In [461]:
# plt.figure(figsize=(30, 20))

# # List of tuples with column names and the number of bins you want for each
# columns_and_bins = [
#     ('Tonnage', 50),
#     ('N_cumulee', 50),
#     ('P_cumulee', 50),
#     ('K_cumulee', 50),
#     ('irrigation_cumulee', 50)  # Add your new column here
# ]

# # Create a subplot for each column
# for i, (column, bins) in enumerate(columns_and_bins, 1):
#     ax = plt.subplot(3, 2, i)  # Adjusted for a 3x2 grid of subplots to accommodate the additional plot
#     data = train_test[column].dropna()  # Drop NA values for cleaner histogram
#     ax.hist(data, bins=bins, color='skyblue', edgecolor='black')

#     # Set the title and labels
#     ax.set_title(f'Histogram of {column}', fontsize=20)
#     ax.set_xlabel(column, fontsize=16)
#     ax.set_ylabel('Frequency', fontsize=16)
    
#     # Add gridlines for better readability
#     ax.grid(True)

#     # Uncomment the following lines to annotate the mean or median
#     # mean_value = data.mean()
#     # ax.axvline(mean_value, color='red', linestyle='dashed', linewidth=2)
#     # ax.text(mean_value, plt.gca().get_ylim()[1]*0.9, f'Mean: {mean_value:.2f}', color='red')

# # Adjust the layout so that all plots fit nicely in the figure canvas
# plt.tight_layout()
# plt.show()

### Pred

In [462]:
# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_cols),
        ('impute', knn_imputer, cols_to_impute)
    ],
    remainder='passthrough'
)

# Create a complete pipeline
pipeline = make_pipeline(preprocessor)

In [463]:
# train = train_test[train_test['is_train']]
# test = train_test[~train_test['is_train']]

# # Separate features and target
# X_train = train.drop(['Tonnage', 'is_train', 'Famille'], axis=1)  # Replace 'target' with your actual target column name
# y_train = train['Tonnage']

# X_test = test.drop(['Tonnage', 'is_train', 'Famille'], axis=1)

In [464]:
train = train_test[train_test['is_train']]
test = train_test[~train_test['is_train']]

# Separate features and target
X = train.drop(['Tonnage', 'is_train', 'Famille'], axis=1)  # Replace 'target' with your actual target column name
y = train['Tonnage']

X_test = test.drop(['Tonnage', 'is_train', 'Famille'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X.columns

In [465]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from xgboost import XGBRegressor


# models = [RandomForestRegressor(), XGBRegressor(), LinearRegression()]

In [466]:
# from sklearn.metrics import r2_score

# # Create the complete pipeline
# for model in models:
#     pipeline = make_pipeline(preprocessor, model)
#     pipeline.fit(X_train, y_train)
#     y_train_pred = pipeline.predict(X_train)

#     mse = mean_squared_error(y_train, y_train_pred)
#     rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r_squared = r2_score(y_train, y_train_pred)
#     std = train_test[train_test['is_train']]['Tonnage'].std()

#     print(f'''
#     ▶️ Model  : {model}
#     ▶️ MSE  : {mse}
#     ▶️ RMSE : {rmse} vs STD  : {std}   
#     ▶️ R²   : {r_squared}
#     ''')

In [467]:
from sklearn.linear_model import LinearRegression

model = LinearRegression

In [468]:
pipeline = make_pipeline(preprocessor, model)

In [469]:
pipeline.fit(X_train, y_train)

c:\Users\DataScience\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute '_validate_params'

In [ ]:
y_train_pred = pipeline.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score

mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
r_squared = r2_score(y_train, y_train_pred)
std = train_test[train_test['is_train']]['Tonnage'].std()


r_squared
print(f'''     
    ▶️ MSE  : {mse}
    ▶️ RMSE : {rmse} vs STD  : {std}   
    ▶️ R²   : {r_squared}
    ''')

In [ ]:
y_val_pred = pipeline.predict(X_val)

In [ ]:
from sklearn.metrics import r2_score

mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
r_squared = r2_score(y_val, y_val_pred)
std = train_test[train_test['is_train']]['Tonnage'].std()


r_squared
print(f'''     
    ▶️ MSE  : {mse}
    ▶️ RMSE : {rmse} vs STD  : {std}   
    ▶️ R²   : {r_squared}
    ''')

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
y_pred = np.array(y_pred)  # Convert to numpy array if not already
assert len(y_pred) == 781, "y_pred must have exactly 781 rows"

# Create a DataFrame. Adjust column names as per the competition's requirement.
# Usually, you will have an ID column and a prediction column.
submission_df = pd.DataFrame({
    'Id': range(1, 782),  # Example: creating a sequence of IDs from 1 to 781
    'Tonnage': y_pred
})

# Export to CSV
csv_file = "submission.csv"
submission_df.to_csv(csv_file, index=False)